# Run ipSAE on Boltz-2 Candidate Structures

In [1]:
import pandas as pd
import os

In [3]:
## Read in candidates data
candidates_df = pd.read_excel('../candidates.xlsx', sheet_name='Antibody Candidates')
antigens_df = pd.read_excel('../candidates.xlsx', sheet_name='Antigens')

## Get the antigen sequence for Nipah Glycoprotein G
# antigen_seq = antigens_df.loc[antigens_df['antigen_id'] == 'nipah_gpG', 'antigen_sequence'].values[0]
antigen_seq = antigens_df.loc[antigens_df['antigen_id'] == 'nipah_gpG_compout', 'antigen_sequence'].values[0]
print(f"Using Antigen Sequence: {antigen_seq}")

Using Antigen Sequence: MPAENKKVRFENTTSDKGKIPSKVIKSYYGTMDIKKINEGLLDSKILSAFNTVIALLGSIVIIVMNIMIIQNYTRSTDNQAVIKDALQGIQQQIKGLADKIGTEIGPKVSLIDTSSTITIPANIGLLGSKISQSTASINENVNEKCKFTLPPLKIHECNISCPNPLPFREYRPQTEGVSNLVGLPNNICLQKTSNQILKPKLISYTLPVVGQSGTCITDPLLAMDEGYFAYSHLERIGSCSRGVSKQRIIGVGEVLDRGDEVPSLFMTNVWTPPNPNTVYHCSAVYNNEFYYVLCAVSTVGDPILNSTYWSGSLMMTRLAVKPKSNGGGYNQHQLALRSIEKGRYDKVMPYGPSGIKQGDTLYFPAVGFLVRTEFKYNDSNCPITKCQYSKPENCRLSMGIRPNSHYILRSGLLKYNLSDGENPKVVFIEISDQRLSIGSPSKIYDSLGQPVFYQASFSWDTMIKFGDVLTVNPLVVNWRNNTVISRPGQSQCPRFNTCPEICWEGVYNDAFLIDRINWISAGVFLDSNQTAENPVFTVFKDNEILYRAQLASEDTNAQKTITNCFLLKNKIWCISLVEIYDTGDNVIRPKLFAVKIPEQCT


In [4]:
def calculate_ipsae(
    pae_file_path,
    structure_file_path,
    pae_cutoff=15.0,
    dist_cutoff=15.0,
):
    """
    Calculate ipSAE and related scores for protein-protein interactions.
    SOURCE: https://github.com/adaptyvbio/nipah_ipsae_pipeline/blob/main/Boltz-IPSAE.ipynb

    Parameters:
    -----------
    pae_file_path : str
        Path to the PAE file (JSON for AF2/AF3, NPZ for Boltz1)
    structure_file_path : str
        Path to the structure file (PDB for AF2, mmCIF for AF3/Boltz1)
    pae_cutoff : float
        Cutoff value for PAE in score calculations
    dist_cutoff : float
        Cutoff value for distance in score calculations

    Returns:
    --------
    dict
        Dictionary containing all calculated scores
    """

    os.system(f"python helper_scripts/ipsae.py {pae_file_path} {structure_file_path} {pae_cutoff} {dist_cutoff}")

    print(f"Reading results from {structure_file_path.replace('.cif',  f'_{int(pae_cutoff)}_{int(dist_cutoff)}.txt')}")

    df = pd.read_csv(structure_file_path.replace('.cif', f'_{int(pae_cutoff)}_{int(dist_cutoff)}.txt'))
    results = {}


    for i, row in df[df.Type=="max"].iterrows():
        chainpair = f"{row['Chn1']}-{row['Chn2']}"

        results[chainpair] = {
            "max": {
                **{col: row[col] for col in df.columns[5:-1]}
            }
        }
        mask = (df['Chn1'] == row['Chn1']) & (df['Chn2'] == row['Chn2']) & (df['Type'] != "max")
        min_vals = df[mask][df.columns[5:-1]].min()
        results[chainpair]["min"] = min_vals.to_dict()

    return results
   

In [7]:
## Loop through each candidate and run the ipsae script
scores_df = pd.DataFrame(
    columns=[
        'antibody_id',
        'min_ipsae',
        'max_ipsae'
        ]
    )

for idx, row in candidates_df.iterrows():
    antibody_id = row['antibody_id']
    print(f"Scoring Boltz-2 outputs for Candidate: {antibody_id}")
    prediction_dir = f'../data/candidates/structures_boltz2_frankenchain/boltz_results_{antibody_id}/predictions/{antibody_id}/'
    stem_name = f'{antibody_id}_model_0'

    pae_file_path = os.path.join(prediction_dir, f'pae_{stem_name}.npz')
    structure_file_path = os.path.join(prediction_dir, f'{stem_name}.cif')

    ## Check if prediction directory exists
    if not os.path.exists(prediction_dir):
        print(f"Prediction directory not found for {antibody_id}, skipping...")
        continue

    try:
        score_results = calculate_ipsae(pae_file_path, structure_file_path, pae_cutoff=15.0, dist_cutoff=15.0)

        ipsae_dict = {
            'antibody_id': antibody_id,
            'min_ipsae': score_results['A-B']['min']['ipSAE'],
            'max_ipsae': score_results['A-B']['max']['ipSAE']
        }

        ipsae_df = pd.DataFrame([ipsae_dict])

        scores_df = pd.concat([scores_df, ipsae_df], ignore_index=True)
    except Exception as e:
        print(f"Error processing {antibody_id}: {e}")
        continue




Scoring Boltz-2 outputs for Candidate: sbio-nipahgpg-001
Reading results from ../data/candidates/structures_boltz2_frankenchain/boltz_results_sbio-nipahgpg-001/predictions/sbio-nipahgpg-001/sbio-nipahgpg-001_model_0_15_15.txt
Scoring Boltz-2 outputs for Candidate: sbio-nipahgpg-002


C:\Users\Colby\AppData\Local\Temp\ipykernel_41948\1312463064.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  scores_df = pd.concat([scores_df, ipsae_df], ignore_index=True)


Reading results from ../data/candidates/structures_boltz2_frankenchain/boltz_results_sbio-nipahgpg-002/predictions/sbio-nipahgpg-002/sbio-nipahgpg-002_model_0_15_15.txt
Scoring Boltz-2 outputs for Candidate: sbio-nipahgpg-003
Reading results from ../data/candidates/structures_boltz2_frankenchain/boltz_results_sbio-nipahgpg-003/predictions/sbio-nipahgpg-003/sbio-nipahgpg-003_model_0_15_15.txt
Scoring Boltz-2 outputs for Candidate: sbio-nipahgpg-004
Reading results from ../data/candidates/structures_boltz2_frankenchain/boltz_results_sbio-nipahgpg-004/predictions/sbio-nipahgpg-004/sbio-nipahgpg-004_model_0_15_15.txt
Scoring Boltz-2 outputs for Candidate: sbio-nipahgpg-005
Reading results from ../data/candidates/structures_boltz2_frankenchain/boltz_results_sbio-nipahgpg-005/predictions/sbio-nipahgpg-005/sbio-nipahgpg-005_model_0_15_15.txt
Scoring Boltz-2 outputs for Candidate: sbio-nipahgpg-008
Reading results from ../data/candidates/structures_boltz2_frankenchain/boltz_results_sbio-nipahg

In [9]:
scores_df

,antibody_id,min_ipsae,max_ipsae
0,sbio-nipahgpg-001,0.005304,0.025983
1,sbio-nipahgpg-002,0.000000,0.005246
2,sbio-nipahgpg-003,0.047468,0.093658
3,sbio-nipahgpg-004,0.012938,0.057574
4,sbio-nipahgpg-005,0.000000,0.005056
...,...,...,...
148,sbio-nipahgpg-155,0.030435,0.063979
149,sbio-nipahgpg-156,0.000000,0.004908
150,sbio-nipahgpg-157,0.042745,0.140499
151,sbio-nipahgpg-158,0.000000,0.010083


In [ ]:
# scores_df.to_csv('../data/candidates/structures_boltz2_frankenchain/boltz2_frankenchain_ipsae_scores.csv', index=False)